<a href="https://colab.research.google.com/github/jetsonmom/chat_ai/blob/main/1229_wether_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio requests


In [ ]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta

def get_current_weather(city):
    # 기상청 API의 nx, ny 좌표 (예: 서울)
    location_coordinates = {
        "서울": (60, 127),
        "부산": (76, 84),
        "대구": (89, 73),
        "인천": (55, 126),
        "광주": (39, 125),
        "대전": (75, 90),
        "울산": (80, 85),
        "세종": (70, 95),
        # 필요한 다른 도시 추가 가능
    }

    if city not in location_coordinates:
        return f"{city}에 대한 정보는 없습니다."

    nx, ny = location_coordinates[city]

    # 기상청 API URL 및 파라미터 설정
    WEATHER_API_KEY = "9dfs5NiWTVeX7OTYlr1XFA"  # 여기에 실제 API 키 입력
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'

    # 현재 시간 계산
    now = datetime.now()

    if now.minute < 40:
        if now.hour == 0:
            yesterday = now - timedelta(days=1)
            base_date = yesterday.strftime("%Y%m%d")
            base_time = "2330"
        else:
            base_date = now.strftime("%Y%m%d")
            base_time = (now - timedelta(hours=1)).strftime("%H30")
    else:
        base_date = now.strftime("%Y%m%d")
        base_time = now.strftime("%H00")

    params = {
        'serviceKey': WEATHER_API_KEY,
        'pageNo': '1',
        'numOfRows': '1000',
        'dataType': 'XML',
        'base_date': base_date,
        'base_time': base_time,
        'nx': nx,
        'ny': ny,
    }

    response = requests.get(url, params=params)
    print("API 응답 상태 코드:", response.status_code)  # 상태 코드 출력
    print("API 응답 내용:", response.content.decode('utf-8'))  # 응답 내용 출력
    if response.status_code == 200:
        root = ET.fromstring(response.content)

        weather_info = {}
        for item in root.findall(".//item"):
            category = item.find('category').text
            value = item.find('obsrValue').text

            if category == 'T1H':  # 현재 기온
                temperature = float(value)
                weather_info['temperature'] = temperature

            elif category == 'REH':  # 습도
                humidity = float(value)
                weather_info['humidity'] = humidity

            elif category == 'SKY':  # 날씨 상태
                weather_description = value
                weather_info['weather_description'] = weather_description

        return (weather_info.get('weather_description', '알 수 없음'),
                weather_info.get('temperature', '알 수 없음'),
                weather_info.get('humidity', '알 수 없음'))

    else:
        return f"API 요청 실패: {response.status_code}"


In [ ]:
import gradio as gr

def chatbot(city):
    weather_info = get_current_weather(city)
    if isinstance(weather_info, tuple):
        description, temperature, humidity = weather_info
        return f"날씨: {description}, 온도: {temperature}°C, 습도: {humidity}%"
    else:
        return weather_info

# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="날씨 챗봇",
    description="원하는 도시의 현재 날씨를 알려드립니다."
)

# 인터페이스 실행
iface.launch()


In [ ]:
# 필요한 라이브러리 설치
!pip install gradio requests

import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import gradio as gr

# 기상청 API의 nx, ny 좌표 (예: 서울)
location_coordinates = {
    "서울": (60, 127),
    "부산": (76, 84),
    "대구": (89, 73),
    "인천": (55, 126),
    "광주": (39, 125),
    "대전": (75, 90),
    "울산": (80, 85),
    "세종": (70, 95),
}

def get_current_weather(city):
    if city not in location_coordinates:
        return f"{city}에 대한 정보는 없습니다."

    nx, ny = location_coordinates[city]

    # 기상청 API URL 및 파라미터 설정
    WEATHER_API_KEY = "YOUR_WEATHER_API_KEY"  # 여기에 실제 API 키 입력
    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'

    # 현재 시간 계산
    now = datetime.now()

    if now.minute < 40:
        if now.hour == 0:
            yesterday = now - timedelta(days=1)
            base_date = yesterday.strftime("%Y%m%d")
            base_time = "2330"
        else:
            base_date = now.strftime("%Y%m%d")
            base_time = (now - timedelta(hours=1)).strftime("%H30")
    else:
        base_date = now.strftime("%Y%m%d")
        base_time = now.strftime("%H00")

    params = {
        'serviceKey': WEATHER_API_KEY,
        'pageNo': '1',
        'numOfRows': '1000',
        'dataType': 'XML',
        'base_date': base_date,
        'base_time': base_time,
        'nx': nx,
        'ny': ny,
    }

    response = requests.get(url, params=params)

    print("API 응답 상태 코드:", response.status_code)  # 상태 코드 출력
    print("API 응답 내용:", response.content.decode('utf-8'))  # 응답 내용 출력

    if response.status_code == 200:
        root = ET.fromstring(response.content)

        weather_info = {}
        for item in root.findall(".//item"):
            category = item.find('category').text
            value = item.find('obsrValue').text

            if category == 'T1H':  # 현재 기온
                temperature = float(value)
                weather_info['temperature'] = temperature

            elif category == 'REH':  # 습도
                humidity = float(value)
                weather_info['humidity'] = humidity

            elif category == 'SKY':  # 날씨 상태
                weather_description = value
                weather_info['weather_description'] = weather_description

        return (weather_info.get('weather_description', '알 수 없음'),
                weather_info.get('temperature', '알 수 없음'),
                weather_info.get('humidity', '알 수 없음'))

    else:
        return f"API 요청 실패: {response.status_code}, 메시지: {response.text}"

def chatbot(city):
    weather_info = get_current_weather(city)
    if isinstance(weather_info, tuple):
        description, temperature, humidity = weather_info
        return f"날씨: {description}, 온도: {temperature}°C, 습도: {humidity}%"
    else:
        return weather_info

# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="날씨 챗봇",
    description="원하는 도시의 현재 날씨를 알려드립니다."
)

# 인터페이스 실행
iface.launch()


In [ ]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import gradio as gr

# 기상청 API의 nx, ny 좌표 (예: 서울)
location_coordinates = {
    "서울": (60, 127),
    "부산": (76, 84),
    "대구": (89, 73),
    "인천": (55, 126),
    "광주": (39, 125),
    "대전": (75, 90),
    "울산": (80, 85),
    "세종": (70, 95),
}

def get_current_weather(city):
    if city not in location_coordinates:
        return f"{city}에 대한 정보는 없습니다."

    nx, ny = location_coordinates[city]

    # 기상청 API URL 및 파라미터 설정
    WEATHER_API_KEY = "bfb40e3e3234f989fc6ad96425b172f8"  # 여기에 실제 API 키 입력
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric&lang=kr"

    # 현재 시간 계산
    now = datetime.now()

    if now.minute < 40:
        if now.hour == 0:
            yesterday = now - timedelta(days=1)
            base_date = yesterday.strftime("%Y%m%d")
            base_time = "2330"
        else:
            base_date = now.strftime("%Y%m%d")
            base_time = (now - timedelta(hours=1)).strftime("%H30")
    else:
        base_date = now.strftime("%Y%m%d")
        base_time = now.strftime("%H00")

    params = {
        'serviceKey': WEATHER_API_KEY,
        'pageNo': '1',
        'numOfRows': '1000',
        'dataType': 'XML',
        'base_date': base_date,
        'base_time': base_time,
        'nx': nx,
        'ny': ny,
    }

    response = requests.get(url, params=params)

    # 응답 상태 코드 및 내용 출력
    print("API 응답 상태 코드:", response.status_code)
    print("API 응답 내용:", response.content.decode('utf-8'))

    if response.status_code == 200:
        root = ET.fromstring(response.content)

        weather_info = {}
        for item in root.findall(".//item"):
            category = item.find('category').text
            value = item.find('obsrValue').text

            if category == 'T1H':  # 현재 기온
                temperature = float(value)
                weather_info['temperature'] = temperature

            elif category == 'REH':  # 습도
                humidity = float(value)
                weather_info['humidity'] = humidity

            elif category == 'SKY':  # 날씨 상태
                weather_description = value
                weather_info['weather_description'] = weather_description

        return (weather_info.get('weather_description', '알 수 없음'),
                weather_info.get('temperature', '알 수 없음'),
                weather_info.get('humidity', '알 수 없음'))

    else:
        return f"API 요청 실패: {response.status_code}, 메시지: {response.text}"

def chatbot(city):
    weather_info = get_current_weather(city)
    if isinstance(weather_info, tuple):
        description, temperature, humidity = weather_info
        return f"날씨: {description}, 온도: {temperature}°C, 습도: {humidity}%"
    else:
        return weather_info

# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="날씨 챗봇",
    description="원하는 도시의 현재 날씨를 알려드립니다."
)

# 인터페이스 실행
iface.launch()


In [ ]:
!pip install requests gradio
import requests

def get_current_weather(city):
    # OpenWeather API 키 입력
    API_KEY = "bfb40e3e3234f989fc6ad96425b172f8"  # 여기에 실제 API 키 입력
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric&lang=kr"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        # 필요한 데이터 추출
        weather_description = data['weather'][0]['description']
        temperature = data['main']['temp']
        humidity = data['main']['humidity']

        return (weather_description, temperature, humidity)
    else:
        return f"API 요청 실패: {response.status_code}, 메시지: {response.text}"
import gradio as gr

def chatbot(city):
    weather_info = get_current_weather(city)
    if isinstance(weather_info, tuple):
        description, temperature, humidity = weather_info
        return f"날씨: {description}, 온도: {temperature}°C, 습도: {humidity}%"
    else:
        return weather_info

# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="날씨 챗봇",
    description="원하는 도시의 현재 날씨를 알려드립니다."
)

# 인터페이스 실행
iface.launch()


Seongnam, Anyang, Hanam, Pocheon, Gangneung

if response.status_code == 200:클라이언트의 요청이 성공적으로 처리되었음을 나타냅니다

In [ ]:
import requests
# Seongnam"과 "Anyang, "Hanam" (하남시)"Pocheon" (포천시)"Gangneung" (강릉시)
def get_current_weather(city):
    # OpenWeather API 키 입력
    API_KEY = "bfb40e3e3234f989fc6ad96425b172f8"  # 여기에 실제 API 키 입력
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric&lang=kr"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        # 필요한 데이터 추출
        weather_description = data['weather'][0]['description']
        temperature = data['main']['temp']
        humidity = data['main']['humidity']

        return (weather_description, temperature, humidity)
    else:
        return f"API 요청 실패: {response.status_code}, 메시지: {response.text}"
import gradio as gr

def chatbot(city):
    weather_info = get_current_weather(city)
    if isinstance(weather_info, tuple):
        description, temperature, humidity = weather_info
        return f"날씨: {description}, 온도: {temperature}°C, 습도: {humidity}%"
    else:
        return weather_info

# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="날씨 챗봇",
    description="원하는 도시의 현재 날씨를 알려드립니다."
)

# 인터페이스 실행
iface.launch()


젯슨맘 쳇봇

In [ ]:
import requests
import gradio as gr

def get_current_weather(city):
    # OpenWeather API 키 입력
    API_KEY = "bfb40e3e3234f989fc6ad96425b172f8"  # 여기에 실제 API 키 입력
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric&lang=kr"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        # 필요한 데이터 추출
        weather_description = data['weather'][0]['description']
        temperature = data['main']['temp']
        humidity = data['main']['humidity']

        return (weather_description, temperature, humidity)
    else:
        return f"API 요청 실패: {response.status_code}, 메시지: {response.text}"

def chatbot(user_input):
    # 사용자 인사말
    greeting = "안녕하세요!"

    # 인사말 처리
    greetings_list = ["안녕", "안녕하세요", "여보세요", "헬로우", "하이"]

    if any(greet in user_input for greet in greetings_list):
        return f"{greeting} 무엇을 도와드릴까요?"

    # 도시 이름 추출 (예: "Hanam, KR" 또는 "Pocheon, KR")
    city_name = user_input.strip() + ", KR"  # 국가 코드 추가

    weather_info = get_current_weather(city_name)

    if isinstance(weather_info, tuple):
        description, temperature, humidity = weather_info
        return f"{greeting}\n날씨: {description}, 온도: {temperature}°C, 습도: {humidity}%"
    else:
        return f"{greeting}\n{weather_info}"

# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="젯슨맘 날씨 챗봇",
    description="원하는 도시의 현재 날씨를 알려드립니다."
)

# 인터페이스 실행
iface.launch()


In [ ]:
import requests
import gradio as gr

def get_current_weather(city):
    # OpenWeather API 키 입력
    API_KEY = "bfb40e3e3234f989fc6ad96425b172f8"  # 여기에 실제 API 키 입력
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric&lang=kr"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        # 필요한 데이터 추출
        weather_description = data['weather'][0]['description']
        temperature = data['main']['temp']
        humidity = data['main']['humidity']

        return (weather_description, temperature, humidity)
    else:
        return f"API 요청 실패: {response.status_code}, 메시지: {response.text}"

def chatbot(user_input):
    # 사용자 인사말
    greeting = "안녕하세요!"

    # 도시 이름 추출 (예: "Hanam, KR" 또는 "Pocheon, KR")
    city_name = user_input.strip() + ", KR"  # 국가 코드 추가

    weather_info = get_current_weather(city_name)

    if isinstance(weather_info, tuple):
        description, temperature, humidity = weather_info
        return f"{greeting}\n{user_input}의 날씨: {description}, 온도: {temperature}°C, 습도: {humidity}%"
    else:
        return f"{greeting}\n{weather_info}"

# Gradio 인터페이스 생성
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="젯슨맘 날씨 챗봇",
    description="원하는 도시의 현재 날씨를 알려드립니다."
)

# 인터페이스 실행
iface.launch()
